In [ ]:
from utils.models import GMF, MLP, NeuFM
from utils.dataset import RatingsDataset
from torch.utils.data import DataLoader

from tqdm.notebook import tqdm

import math

## Load

In [ ]:
run = 4

In [3]:
users = 3974
movies = 3564

train_dataloader = DataLoader(
    RatingsDataset(
        "train_data/train.csv",
        "user_id",
        "movie_id",
        "rating",
    ), 
    batch_size=1024,
    num_workers=8,
    shuffle=True,
)
val_dataloader = DataLoader(
    RatingsDataset(
        "train_data/validation.csv",
        "user_id",
        "movie_id",
        "rating",
    ),
    batch_size=1024,
    num_workers=4,
    shuffle=True,
)

In [4]:
import torch
import torch.nn as nn
import torch.optim as opt

In [5]:
torch.manual_seed(2)

In [6]:
# model = NeuFM(
#     GMF(users, movies, 8),
#     MLP(users, movies, 16, [16, 8]),
#     alpha=0.5,
# ).cuda()

model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

model

NeuFM(
  (h): Linear(in_features=24, out_features=1, bias=False)
  (gmf): GMF(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (h): Identity()
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Identity()
  )
)

## Train

In [7]:
criterion = nn.MSELoss().cuda()
optimizer = opt.Adam(model.parameters(), lr=0.0005, amsgrad=True)

prev_val_loss = math.inf
for epoch in tqdm(range(20)):
    n_batches = len(train_dataloader)
    avg_loss = 0
    val_loss = 0
    
    # Train step
    for i_batch, (vus, vis, rs) in enumerate(train_dataloader):
        vus = vus.cuda()
        vis = vis.cuda()
        rs = rs.cuda()
        
        optimizer.zero_grad()
        y_hat = model(vus, vis)
        
        loss = criterion(y_hat, rs)
        loss.backward()
        optimizer.step()
        
        avg_loss += math.sqrt(float(loss.detach().cpu()))
        
    # Val step
    with torch.no_grad():
        for val_vus, val_vis, val_rs in val_dataloader:
            val_vus = val_vus.cuda()
            val_vis = val_vis.cuda()
            val_rs = val_rs.cuda()

            val_pred = model(val_vus, val_vis)
            val_loss += math.sqrt(float(criterion(val_pred, val_rs).detach().cpu()))
                
    val_loss /= len(val_dataloader)
    print(f"epoch: {epoch}, train_loss: {avg_loss/n_batches}, val_loss: {val_loss}")
    
    if val_loss < prev_val_loss:
        prev_val_loss = val_loss
        torch.save(
            model.state_dict(), 
            f"/home/nubol23/Documents/NCF_weights/run_{run}/{epoch}-{val_loss}.pt"
        )

  0%|          | 0/20 [00:00<?, ?it/s]

epoch: 0, train_loss: 2.5820931031275363, val_loss: 1.0347566846324814
epoch: 1, train_loss: 0.9638427902319918, val_loss: 0.935728844367984
epoch: 2, train_loss: 0.9159413716897323, val_loss: 0.9147839430213504
epoch: 3, train_loss: 0.899503716831985, val_loss: 0.9035387571123404
epoch: 4, train_loss: 0.887214823760093, val_loss: 0.8952150905262272
epoch: 5, train_loss: 0.8768470997872058, val_loss: 0.8889285789026878
epoch: 6, train_loss: 0.8685040389266401, val_loss: 0.8852713042610598
epoch: 7, train_loss: 0.8622297532506206, val_loss: 0.8829272402397081
epoch: 8, train_loss: 0.8571892774943336, val_loss: 0.8808453327493494
epoch: 9, train_loss: 0.8527954632992225, val_loss: 0.8794175176793041
epoch: 10, train_loss: 0.8485364062202472, val_loss: 0.8779943497062457
epoch: 11, train_loss: 0.8441940447671465, val_loss: 0.8764729657811058
epoch: 12, train_loss: 0.8395804849169238, val_loss: 0.8752950169679902
epoch: 13, train_loss: 0.8347677517465643, val_loss: 0.8737750775447649
epoch

## Predict

In [11]:
trained_weights = "19-0.8679568715990608.pt"

In [13]:
# trained_model = NeuFM(
#     GMF(users, movies, 8),
#     MLP(users, movies, 16, [16, 8]),
#     alpha=0.5,
# )

# trained_model = NeuFM(
#     GMF(users, movies, 8),
#     MLP(users, movies, 32, [32, 16, 8]),
#     alpha=0.5
# )

trained_model = NeuFM(
    GMF(users, movies, 16),
    MLP(users, movies, 16, [16, 8]),
    alpha=0.5,
).cuda()

trained_model.load_state_dict(
    torch.load(
        f"/home/nubol23/Documents/NCF_weights/run_{run}/{trained_weights}"
    )
)
trained_model.cuda().eval()

NeuFM(
  (h): Linear(in_features=24, out_features=1, bias=False)
  (gmf): GMF(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (h): Identity()
  )
  (mlp): MLP(
    (P): Embedding(3974, 16)
    (Q): Embedding(3564, 16)
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): Linear(in_features=16, out_features=8, bias=True)
    )
    (h): Identity()
  )
)

In [14]:
test_dataloader = DataLoader(
    RatingsDataset(
        "train_data/test.csv",
        "user_id",
        "movie_id",
    ), 
    batch_size=1024,
    num_workers=12
)

In [15]:
test_predictions = []

for vus, vis in test_dataloader:
    vus = vus.cuda()
    vis = vis.cuda()

    pred = torch.clip(trained_model(vus, vis), 1, 5).cpu().ravel().tolist()
    test_predictions += pred

In [16]:
import pandas as pd

test_csv = pd.read_csv("../../data/test_data.csv")

In [17]:
out_df = pd.DataFrame.from_dict(
    {
        "id": list(test_csv["id"]),
        "rating": test_predictions
    }
)

out_df.head()

,id,rating
0,0,3.146729
1,1,2.996804
2,2,2.907517
3,3,3.363933
4,4,3.030551


In [ ]:
# out_df.to_csv(f"outputs_csv/neumf_5.csv", index=False)